## Integration of Postgres with Pandas
Pandas is a powerful Python Data Library which is used to process as well as analyze the data. It have robust APIs to work with databases.

Here are the steps involved to use Pandas to work with databases like Postgres.
* We need to make sure `pandas`, `psycopg2-binary` as well as `sqlalchemy` installed using `pip`.
* Pandas uses `sqlalchemy` to interact with database tables based on the connection url.
* `sqlalchemy` is the most popular ORM to hide the complexity of connecting to the databases using our applications.
* Here are the examples using Pandas. 
  * Read from file, Process and Write to Postgres Database Table using Pandas. 
  * Read from Postgres Database Table using Pandas for the validation.

In [1]:
#!pip install psycopg2-binary

In [2]:
#!pip install pandas

In [3]:
#!pip install sqlalchemy

In [4]:
import pandas as pd

In [5]:
columns = ['order_id', 'order_date', 'order_customer_id', 'order_status']

In [6]:
import json

In [7]:
#schemas = json.load(open('../../data/retail_db/schemas.json'))
import os
schemas = json.load (open (os.sep.join ([os.getcwd (), "data-engineering-on-gcp", "data", "retail_db", "schemas.json"])))

In [8]:
sorted(schemas['orders'], key=lambda col: col['column_position'])

[{'column_name': 'order_id', 'data_type': 'integer', 'column_position': 1},
 {'column_name': 'order_date', 'data_type': 'string', 'column_position': 2},
 {'column_name': 'order_customer_id',
  'data_type': 'timestamp',
  'column_position': 3},
 {'column_name': 'order_status', 'data_type': 'string', 'column_position': 4}]

In [9]:
columns = [col['column_name'] for col in sorted(schemas['orders'], key=lambda col: col['column_position'])]

In [10]:
columns

['order_id', 'order_date', 'order_customer_id', 'order_status']

In [11]:
#pd.read_csv?

In [12]:
#orders = pd.read_csv('../../data/retail_db/orders/part-00000', names=columns)
orders = pd.read_csv (os.sep.join ([os.getcwd (), "data-engineering-on-gcp", "data", "retail_db", "orders", "part-00000"]), names=columns)

In [13]:
orders

,order_id,order_date,order_customer_id,order_status
0,1,2013-07-25 00:00:00.0,11599,CLOSED
1,2,2013-07-25 00:00:00.0,256,PENDING_PAYMENT
2,3,2013-07-25 00:00:00.0,12111,COMPLETE
3,4,2013-07-25 00:00:00.0,8827,CLOSED
4,5,2013-07-25 00:00:00.0,11318,COMPLETE
...,...,...,...,...
68878,68879,2014-07-09 00:00:00.0,778,COMPLETE
68879,68880,2014-07-13 00:00:00.0,1117,COMPLETE
68880,68881,2014-07-19 00:00:00.0,2518,PENDING_PAYMENT
68881,68882,2014-07-22 00:00:00.0,10000,ON_HOLD


In [14]:
type(orders)

pandas.core.frame.DataFrame

In [15]:
orders.shape

(68883, 4)

In [26]:
daily_status_count = orders. \
    groupby(['order_date', 'order_status'])['order_id']. \
    agg(order_count='count'). \
    reset_index()

In [27]:
daily_status_count

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [28]:
daily_status_count.to_sql?

Signature:
daily_status_count.to_sql(
    name: 'str',
    con,
    schema: 'str | None' = None,
    if_exists: 'str' = 'fail',
    index: 'bool_t' = True,
    index_label: 'IndexLabel' = None,
    chunksize: 'int | None' = None,
    dtype: 'DtypeArg | None' = None,
    method: 'str | None' = None,
) -> 'int | None'
Docstring:
Write records stored in a DataFrame to a SQL database.

Databases supported by SQLAlchemy [1]_ are supported. Tables can be
newly created, appended to, or overwritten.

Parameters
----------
name : str
    Name of SQL table.
con : sqlalchemy.engine.(Engine or Connection) or sqlite3.Connection
    Using SQLAlchemy makes it possible to use any DB supported by that
    library. Legacy support is provided for sqlite3.Connection objects. The user
    is responsible for engine disposal and connection closure for the SQLAlchemy
    connectable See `here                 <https://docs.sqlalchemy.org/en/13/core/connections.html>`_.

schema : str, optional
    Specify the s

In [29]:
'''
host = '104.197.141.244'
port = 5432
database = 'itversity_retail_db'
user = 'itversity_retail_user'
password = 'itversity'
'''
host = "35.232.183.213"#outgoing IP address of the GCP PostgreSQL Server
port = 5432
database = "udemy_retail_db"#The name of the Postgres db created in the Postgres server by signing in with psql from command line
user = "udemy_retail_user"#The username of the Postgres db user created in the Postgres server by signing in with psql from command line
password = "udemy_retail_dbpassword"#The password of the Postgres db created for the user when signing in with psql from command line

In [30]:
conn_uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [31]:
daily_status_count.to_sql(
    'daily_status_count',
    conn_uri,
    if_exists='replace',
    index=False
)

203

In [34]:
pd.read_sql?

Signature:
pd.read_sql(
    sql,
    con,
    index_col: 'str | list[str] | None' = None,
    coerce_float: 'bool' = True,
    params=None,
    parse_dates=None,
    columns: 'list[str] | None' = None,
    chunksize: 'int | None' = None,
) -> 'DataFrame | Iterator[DataFrame]'
Docstring:
Read SQL query or database table into a DataFrame.

This function is a convenience wrapper around ``read_sql_table`` and
``read_sql_query`` (for backward compatibility). It will delegate
to the specific function depending on the provided input. A SQL query
will be routed to ``read_sql_query``, while a database table name will
be routed to ``read_sql_table``. Note that the delegated function might
have more specific notes about their functionality not listed here.

Parameters
----------
sql : str or SQLAlchemy Selectable (select or text object)
    SQL query to be executed or a table name.
con : SQLAlchemy connectable, str, or sqlite3 connection
    Using SQLAlchemy makes it possible to use any DB suppor

In [40]:
dsc = pd.read_sql(
    'daily_status_count',
    conn_uri
)

In [37]:
dsc.shape

(3203, 3)

In [38]:
dsc

,order_date,order_status,order_count
0,2013-07-25 00:00:00.0,CANCELED,1
1,2013-07-25 00:00:00.0,CLOSED,20
2,2013-07-25 00:00:00.0,COMPLETE,42
3,2013-07-25 00:00:00.0,ON_HOLD,5
4,2013-07-25 00:00:00.0,PAYMENT_REVIEW,3
...,...,...,...
3198,2014-07-24 00:00:00.0,PAYMENT_REVIEW,2
3199,2014-07-24 00:00:00.0,PENDING,23
3200,2014-07-24 00:00:00.0,PENDING_PAYMENT,37
3201,2014-07-24 00:00:00.0,PROCESSING,20


In [44]:
dsc = pd.read_sql(
    '''
        SELECT order_status, sum(order_count) AS order_count FROM daily_status_count
        GROUP BY 1
        ORDER BY 2 DESC
    ''',
    conn_uri
)

In [45]:
dsc

,order_status,order_count
0,COMPLETE,22899.0
1,PENDING_PAYMENT,15030.0
2,PROCESSING,8275.0
3,PENDING,7610.0
4,CLOSED,7556.0
5,ON_HOLD,3798.0
6,SUSPECTED_FRAUD,1558.0
7,CANCELED,1428.0
8,PAYMENT_REVIEW,729.0
